### 1. Importação das bibliotecas

In [ ]:
import requests
import pandas as pd
import sqlite3
from plyer import notification
from datetime import datetime
import os

### 2. Função de criação do alerta

In [ ]:
def enviar_alerta(nivel, base, etapa):
    if nivel == 1:
        alerta = 'Baixo'
    elif nivel == 2:
        alerta = 'Médio'
    else:
        alerta = 'Alto'

    titulo = f"Alerta {alerta}"
    mensagem = f"Falha no carregamento da base {base} na etapa {etapa} \n{datetime.now()}"
    notification.notify(
        title=titulo,
        message=mensagem,
        app_name="etl_pipeline",
        timeout=10
    )

### 3. Obtendo dados da API

In [ ]:
def obter_dados_da_api(url):
    resposta = requests.get(url)
    if resposta.status_code == 200:
        return resposta.json()
    else:
        enviar_alerta(3, url, "Extração")
        print(f"ERRO: Falha ao buscar dados da API. Código de status: {resposta.status_code}")
        return None

### 4. Extração todas as páginas de uma API paginada

In [ ]:
def extrair_todas_paginas(base_url):
    resultados = []
    url = base_url
    while url:
        dados = obter_dados_da_api(url)
        if dados:
            resultados.extend(dados['results'])
            url = dados.get('next')
        else:
            break
    return resultados

### 5. Extração dos dados de pokemons

In [ ]:
def extrair_pokemons(limite=50):
    url = f'https://pokeapi.co/api/v2/pokemon?limit={limite}'
    dados = obter_dados_da_api(url)
    pokemons = []
    
    for item in dados['results']:
        dados_pokemon = obter_dados_da_api(item['url'])
        if dados_pokemon:
            habilidades = [int(habilidade['ability']['url'].split('/')[-2]) for habilidade in dados_pokemon['abilities']]
            tipos = [int(tipo['type']['url'].split('/')[-2]) for tipo in dados_pokemon['types']]
            pokemons.append({
                'id': dados_pokemon['id'],
                'nome': dados_pokemon['name'],
                'experiencia_base': dados_pokemon['base_experience'],
                'altura': dados_pokemon['height'],
                'peso': dados_pokemon['weight'],
                'id_habilidade_1': habilidades[0] if len(habilidades) > 0 else None,
                'id_habilidade_2': habilidades[1] if len(habilidades) > 1 else None,
                'id_tipo_1': tipos[0] if len(tipos) > 0 else None,
                'id_tipo_2': tipos[1] if len(tipos) > 1 else None
            })
    
    # Converte as colunas id_tipo_1 e id_tipo_2 para INT, garantindo que None seja tratado como NaN e depois convertendo para None
    pokemon_df = pd.DataFrame(pokemons)
    pokemon_df['id_tipo_1'] = pokemon_df['id_tipo_1'].astype('Int64')
    pokemon_df['id_tipo_2'] = pokemon_df['id_tipo_2'].astype('Int64')
    
    return pokemon_df

### 6. Extração dos dados de habilidades

In [ ]:
def extrair_habilidades():
    url = 'https://pokeapi.co/api/v2/ability?limit=1000'
    dados = obter_dados_da_api(url)
    habilidades = []
    
    for item in dados['results']:
        dados_habilidade = obter_dados_da_api(item['url'])
        if dados_habilidade:
            habilidades.append({
                'id': dados_habilidade['id'],
                'nome': dados_habilidade['name'],
                'efeito': dados_habilidade['effect_entries'][0]['effect'] if dados_habilidade['effect_entries'] else None
            })
    
    return pd.DataFrame(habilidades)

### 7. Extração dos dados de tipos

In [ ]:
def extrair_tipos():
    url = 'https://pokeapi.co/api/v2/type'
    dados = obter_dados_da_api(url)
    tipos = []
    
    for item in dados['results']:
        dados_tipo = obter_dados_da_api(item['url'])
        if dados_tipo:
            tipos.append({
                'id': dados_tipo['id'],
                'nome': dados_tipo['name']
            })
    
    return pd.DataFrame(tipos)

### 8. Função para criar (CREATE) e popular (INSERT INTO) o banco de dados

In [ ]:
def criar_e_popular_bd(pokemon_df, habilidades_df, tipos_df, db_name='../pokeapi.db'):
    # Remover banco de dados existente, se houver
    if os.path.exists(db_name):
        os.remove(db_name)
    
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Criar tabelas
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS pokemons (
        id INTEGER PRIMARY KEY,
        nome TEXT,
        experiencia_base INTEGER,
        altura INTEGER,
        peso INTEGER,
        id_habilidade_1 INTEGER,
        id_habilidade_2 INTEGER,
        id_tipo_1 INTEGER,
        id_tipo_2 INTEGER,
        FOREIGN KEY (id_habilidade_1) REFERENCES habilidades(id),
        FOREIGN KEY (id_habilidade_2) REFERENCES habilidades(id),
        FOREIGN KEY (id_tipo_1) REFERENCES tipos(id),
        FOREIGN KEY (id_tipo_2) REFERENCES tipos(id)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS habilidades (
        id INTEGER PRIMARY KEY,
        nome TEXT,
        efeito TEXT
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS tipos (
        id INTEGER PRIMARY KEY,
        nome TEXT
    )
    ''')
    
    # Inserir dados nas tabelas
    pokemon_df.to_sql('pokemons', conn, if_exists='append', index=False)
    habilidades_df.to_sql('habilidades', conn, if_exists='append', index=False)
    tipos_df.to_sql('tipos', conn, if_exists='append', index=False)
    
    conn.commit()
    conn.close()

### 9. Executando o ETL

In [ ]:
try:
    pokemons_df = extrair_pokemons(1302)
    habilidades_df = extrair_habilidades()
    tipos_df = extrair_tipos()
    
    criar_e_popular_bd(pokemons_df, habilidades_df, tipos_df)
    print("Dados extraídos e carregados no banco de dados com sucesso!")
except Exception as e:
    enviar_alerta(3, "Processo ETL", "Geral")
    print(f"Erro no processo ETL: {e}")

### 10. Exemplo de erro na extração da base

In [ ]:
obter_dados_da_api("https://pokeapi.co/api/v2/pokemon/1/v2")

### 11. Função para executar uma consulta e retornar um DataFrame

In [ ]:
def executar_consulta(query, db_name='../pokeapi.db'):
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

### 12. Visualizando as tabelas

#### 12.1. Pokémons

In [ ]:
query_pokemons = "SELECT * FROM pokemons LIMIT 10"
df_pokemons = executar_consulta(query_pokemons)
df_pokemons

#### 12.2. Habilidades

In [ ]:
query_habilidades = "SELECT * FROM habilidades WHERE id IN (19, 34, 44, 50, 65, 66, 67, 94)"
df_habilidades = executar_consulta(query_habilidades)
print(df_habilidades)

#### 12.3. Tipos

In [ ]:
query_tipos = "SELECT * FROM tipos WHERE id IN (3.0, 4.0, 7, 10, 11, 12)"
df_tipos = executar_consulta(query_tipos)
print(df_tipos)

#### 12.4. Join

In [ ]:
query_pokemon_habilidades_tipos = """
SELECT p.nome AS nome, p.experiencia_base AS experiencia_base, h1.nome AS habilidade_1, h2.nome AS habilidade_2, t1.nome AS tipo_1, t2.nome AS tipo_2
FROM pokemons p
LEFT JOIN habilidades h1 ON p.id_habilidade_1 = h1.id
LEFT JOIN habilidades h2 ON p.id_habilidade_2 = h2.id
LEFT JOIN tipos t1 ON p.id_tipo_1 = t1.id
LEFT JOIN tipos t2 ON p.id_tipo_2 = t2.id
ORDER BY p.experiencia_base DESC
LIMIT 10
"""
df_pokemon_habilidades_tipos = executar_consulta(query_pokemon_habilidades_tipos)
df_pokemon_habilidades_tipos

### 13. Hipóteses

In [ ]:
# Importando as bibliotecas de visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Retornando consultas com todos os dados das tabelas
query_pokemons = "SELECT * FROM pokemons"
df_pokemons = executar_consulta(query_pokemons)

query_habilidades = "SELECT * FROM habilidades"
df_habilidades = executar_consulta(query_habilidades)

query_tipos = "SELECT * FROM tipos"
df_tipos = executar_consulta(query_tipos)

#### 13.1. Alguns tipos de Pokémons tendem a ter uma experiência base mais alta do que outros

In [ ]:
# Juntando dados de pokemons e tipos
df_pokemons_tipos = df_pokemons.merge(df_tipos, left_on='id_tipo_1', right_on='id', suffixes=('_pokemon', '_tipo'))

# Plotando o gráfico violin
plt.figure(figsize=(14, 6))
sns.violinplot(data=df_pokemons_tipos, x='nome_tipo', y='experiencia_base')
plt.title('Distribuição da Experiência Base por Tipo de Pokémon')
plt.xlabel('Tipo de Pokémon')
plt.ylabel('Experiência Base')
plt.xticks(rotation=45)
plt.show()

#### 13.2. A altura e o peso dos Pokémons variam significativamente entre diferentes tipos

In [ ]:
# Criar uma figura com duas subplots lado a lado
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Gráfico de Altura Média por Tipo de Pokémon
sns.barplot(ax=axes[0], data=df_pokemons_tipos, x='nome_tipo', y='altura')
axes[0].set_title('Altura Média por Tipo de Pokémon')
axes[0].set_xlabel('Tipo de Pokémon')
axes[0].set_ylabel('Altura Média')
axes[0].tick_params(axis='x', rotation=45)

# Gráfico de Peso Médio por Tipo de Pokémon
sns.barplot(ax=axes[1], data=df_pokemons_tipos, x='nome_tipo', y='peso')
axes[1].set_title('Peso Médio por Tipo de Pokémon')
axes[1].set_xlabel('Tipo de Pokémon')
axes[1].set_ylabel('Peso Médio')
axes[1].tick_params(axis='x', rotation=45)

# Ajustar o layout para evitar sobreposição
plt.tight_layout()

# Mostrar a figura com os gráficos
plt.show()

In [ ]:
# Consulta SQL para trazer a média da altura e a média do peso por tipo de Pokémon
query_media_altura_peso_por_tipo = """
SELECT 
    t.nome AS tipo,
    AVG(p.altura) AS altura_media,
    AVG(p.peso) AS peso_medio
FROM pokemons p
JOIN tipos t ON p.id_tipo_1 = t.id
GROUP BY t.nome
"""

# Executar a consulta
df_media_altura_peso = executar_consulta(query_media_altura_peso_por_tipo)

# Mostrar os resultados
print(df_media_altura_peso)